In [26]:
from configparser import ConfigParser, ExtendedInterpolation

import h5py
import numpy as np
import matplotlib.pylab as plt

from spikelib.fitting import fit_temp_sta
from spikelib.utils import check_groups

%matplotlib notebook

config = ConfigParser(interpolation=ExtendedInterpolation())
config.read('../../config.ini')

['../../config.ini']

# General information about input output

In [27]:
# Data files
processed_file = config['FILES']['processed']

# intensities = ['nd2-255', 'nd3-255', 'nd4-255', 'nd5-255']
with h5py.File(processed_file, 'r') as f:
    intensities = list(f['/sta'].keys())
print(f'intensities: {intensities}')

intensities: ['nd3-255']


In [7]:
temp_group = '/sta/{}/temporal/raw/'
spatial_group = '/sta/{}/spatial/char/'
valid = '/sta/{}/valid/'



# Load Data

In [8]:
# Load data

def load_data(processed_file, intensity):
    raw_dtype = np.dtype([('id', 'U18'), ('temp_raw', 'f8',(30,)), ('snr', 'f8')])
    # Groups for raw data in hdf5 file
    temp_group = '/sta/{}/temporal/raw/'.format(intensity)
    spatial_group = '/sta/{}/spatial/char/'.format(intensity)
    valid = '/sta/{}/valid/'.format(intensity)


    with h5py.File(processed_file, 'r') as pfile:
        print(pfile[spatial_group].attrs['col_name'])
        temporal_sta = np.zeros(len(pfile[temp_group]), dtype=raw_dtype)
        for kidx, kkey in enumerate(pfile[temp_group]):
            temporal_sta[kidx]['id'] = kkey
            temporal_sta[kidx]['temp_raw'] = pfile[temp_group][kkey][...]
            temporal_sta[kidx]['snr'] = pfile[spatial_group][kkey][-2]
        fps = pfile[temp_group].attrs['fps'][...]
        time_raw = pfile[temp_group].attrs['time']
    return temporal_sta, time_raw

In [12]:
temporal_sta, time_raw = load_data(processed_file, intensities[0])

upsampling = 30
nsample = time_raw.size
time_fit = np.linspace(time_raw[0], time_raw[-1], nsample*upsampling, endpoint=True)  

angle,a,b,x,y,snr,frame


# Fitting for all units

In [13]:
fit_dtype = np.dtype([('id', 'U18'),
                     ('parameters', 'f8',(5,)),
                     ('temp_fit', 'f8', (nsample*upsampling,)),
                    ])

In [14]:
temp_fit = np.zeros((len(temporal_sta)), dtype=fit_dtype)
no_ajustados = []

for kidx, kcell in enumerate(temporal_sta):
    temp_to_fit = kcell['temp_raw']
    params, tmp_fit = fit_temp_sta(temp_to_fit, time_raw, time_fit)
    
    temp_fit[kidx]['id'] = kcell['id']
    parameters = [params[key] for key in ['amp1', 'amp2', 'tau1', 'tau2', 'n']]
    temp_fit[kidx]['parameters'] = np.asarray(parameters)
    temp_fit[kidx]['temp_fit'] = tmp_fit

/home/vnc/software/spikelib/spikelib/fitting.py:36: RuntimeWarning: divide by zero encountered in power
  gauss1 = amp1 * (time/tau1)**order * np.exp(-order * (time/tau1-1))
/home/vnc/software/spikelib/spikelib/fitting.py:37: RuntimeWarning: divide by zero encountered in power
  gauss2 = amp2 * (time/tau2)**order * np.exp(-order * (time/tau2-1))
/home/vnc/software/spikelib/spikelib/fitting.py:38: RuntimeWarning: invalid value encountered in subtract
  model = gauss1 - gauss2


## Save to hdf5

In [24]:
def save_data(processed_file, intensity, temp_fit, time_fit):
    # Groups for fit data in hdf5 file
    temp_fit_group = '/sta/{}/temporal/fit/'.format(intensity)
    temp_pars_group = '/sta/{}/temporal/fit_params/'.format(intensity)

    with h5py.File(processed_file, 'r+') as pfile:
        check_groups(pfile, [temp_fit_group, temp_pars_group])      
        for kunit in temp_fit:
            key = kunit['id']
            if key in pfile[temp_fit_group]:
                pfile[temp_fit_group+key][...] = kunit['temp_fit']
            else:
                pfile[temp_fit_group].create_dataset(key, data=kunit['temp_fit'], 
                                                         dtype=np.float, compression="gzip")
            if key in pfile[temp_pars_group]:
                pfile[temp_pars_group+key][...] = kunit['parameters']
            else:
                pfile[temp_pars_group].create_dataset(key, data=kunit['parameters'],
                                                          dtype=np.float, compression="gzip")            

        pfile[temp_fit_group].attrs['time'] = time_fit
        pfile[temp_pars_group].attrs['col_name'] = u'amp1,amp2,tau1,tau2,n'
        pfile[temp_pars_group].attrs['info'] = \
            u'parameters are for a two cascade filter described by Chichilnisky2001'


In [25]:
save_data(processed_file, intensities[0], temp_fit, time_fit)